This notebook demonstrates the core concepts of the Bermuda library using test triangle data. We'll explore triangles, cells, slicing, and Ledger-specific conventions.

## Setup and Data Loading

First, let's set up our environment and load the test triangle data.

In [13]:
!pip freeze | grep bermuda

bermuda-ledger==2.1.15


In [11]:
import bermuda  as tri
import altair as alt
from datetime import date, datetime
import os
import urllib.request

# Enable HTML rendering for Altair charts
alt.renderers.enable("html")

RendererRegistry.enable('html')

In [38]:
# Load test triangle data - download if not present
data_file = "data/solm.gl-general.trib"
data_url = "https://ds-ledger-bonneville-data.s3.us-east-2.amazonaws.com/raw/solm/2025/trib/solm.gl-general.trib"

if not os.path.exists(data_file):
    print(f"Downloading test triangle data from {data_url}...")
    os.makedirs("data", exist_ok=True)
    urllib.request.urlretrieve(data_url, data_file)
    print("Download complete.")
else:
    print(f"Using existing data file: {data_file}")

# Load the triangle from the binary trib file
test_triangle = tri.binary_to_triangle('./data/gl.trib')
print("\nTest Triangle loaded successfully:")
print(test_triangle)

Using existing data file: data/solm.gl-general.trib

Test Triangle loaded successfully:
       Cumulative Triangle 


 Number of slices:  1 
 Number of cells:  406 
 Triangle category:  Regular 
 Experience range:  1992-01-01/2019-12-31 
 Experience resolution:  12 
 Evaluation range:  1992-12-31/2019-12-31 
 Evaluation resolution:  12 
 Dev Lag range:  0.0 - 324.0 months 
 Fields: 
   paid_loss
   reported_loss
 Optional Fields: 
   earned_premium (73.9% coverage)
 Common Metadata: 
   risk_basis  Accident 
   reinsurance_basis  Gross 
   loss_definition  Loss+DCC 
   line_of_busines  GL 




## 1. Triangle Overview and Data Completeness

Let's start by examining the overall structure of our triangle and visualizing data completeness. This gives us a bird's eye view of what we're working with.

In [39]:
display(test_triangle)

Cumulative Triangle 


 Number of slices:  1 
 Number of cells:  406 
 Triangle category:  Regular 
 Experience range:  1992-01-01/2019-12-31 
 Experience resolution:  12 
 Evaluation range:  1992-12-31/2019-12-31 
 Evaluation resolution:  12 
 Dev Lag range:  0.0 - 324.0 months 
 Fields: 
   paid_loss
   reported_loss
 Optional Fields: 
   earned_premium (73.9% coverage)
 Common Metadata: 
   risk_basis  Accident 
   reinsurance_basis  Gross 
   loss_definition  Loss+DCC 
   line_of_busines  GL

In [40]:
# Plot data completeness - this shows the "shape" of our triangle
completeness_chart = test_triangle.plot_data_completeness()
completeness_chart

alt.Chart(...)

In [43]:
test_triangle.right_edge.plot_data_completeness()

alt.Chart(...)

## 2. Understanding Triangle Components

Now let's dive into the individual components that make up our triangle.

### 2.1 Periods and Evaluation Dates

Triangles are built from periods (accident/occurrence periods) and evaluation dates.

In [16]:
# Examine the periods in our triangle
print("Experience Periods (first 10):")
for i, period in enumerate(test_triangle.periods[:10]):
    print(f"  {i+1}: {period[0]} to {period[1]}")
print(f"  ... and {len(test_triangle.periods)-10} more periods")

Experience Periods (first 10):
  1: 1997-01-01 to 1997-12-31
  2: 1998-01-01 to 1998-12-31
  3: 1999-01-01 to 1999-12-31
  4: 2000-01-01 to 2000-12-31
  5: 2001-01-01 to 2001-12-31
  6: 2002-01-01 to 2002-12-31
  7: 2003-01-01 to 2003-12-31
  8: 2004-01-01 to 2004-12-31
  9: 2005-01-01 to 2005-12-31
  10: 2006-01-01 to 2006-12-31
  ... and 18 more periods


In [17]:
# Examine evaluation dates
print("Evaluation Dates (first 10):")
for i, eval_date in enumerate(test_triangle.evaluation_dates[:10]):
    print(f"  {i+1}: {eval_date}")
print(f"  ... and {len(test_triangle.evaluation_dates)-10} more evaluation dates")

Evaluation Dates (first 10):
  1: 1997-12-31
  2: 1998-12-31
  3: 1999-12-31
  4: 2000-12-31
  5: 2001-12-31
  6: 2002-12-31
  7: 2003-12-31
  8: 2004-12-31
  9: 2005-12-31
  10: 2006-12-31
  ... and 18 more evaluation dates


### 2.2 Cumulative vs Incremental Cells

Cells are the fundamental building blocks of triangles. They can be cumulative or incremental.

In [45]:
# Examine a cumulative cell
print("Example Cumulative Cell:")
cumulative_cell = test_triangle[0]
print(cumulative_cell)
print(f"\nCell type: {type(cumulative_cell).__name__}")
print(f"Period: {cumulative_cell.period_start} to {cumulative_cell.period_end}")
print(f"Evaluation Date: {cumulative_cell.evaluation_date}")
print(f"Values: {cumulative_cell.values}")

Example Cumulative Cell:
CumulativeCell(period_start=datetime.date(1992, 1, 1), period_end=datetime.date(1992, 12, 31), evaluation_date=datetime.date(1992, 12, 31), values={'paid_loss': 40386429.000000015, 'reported_loss': 169220997.99999997}, metadata=Metadata(risk_basis='Accident', country=None, currency=None, reinsurance_basis='Gross', loss_definition='Loss+DCC', per_occurrence_limit=None, details={'line_of_busines': 'GL'}, loss_details={}))

Cell type: CumulativeCell
Period: 1992-01-01 to 1992-12-31
Evaluation Date: 1992-12-31
Values: {'paid_loss': 40386429.000000015, 'reported_loss': 169220997.99999997}


In [44]:
# Convert to incremental and examine the same cell
incremental_triangle = test_triangle.to_incremental()
print("Example Incremental Cell:")
incremental_cell = incremental_triangle[0]
print(incremental_cell)
print(f"\nCell type: {type(incremental_cell).__name__}")
print(f"Previous Evaluation Date: {incremental_cell.prev_evaluation_date}")

Example Incremental Cell:
IncrementalCell(period_start=datetime.date(1992, 1, 1), period_end=datetime.date(1992, 12, 31), evaluation_date=datetime.date(1992, 12, 31), values={'paid_loss': 40386429.000000015, 'reported_loss': 169220997.99999997}, metadata=Metadata(risk_basis='Accident', country=None, currency=None, reinsurance_basis='Gross', loss_definition='Loss+DCC', per_occurrence_limit=None, details={'line_of_busines': 'GL'}, loss_details={}), prev_evaluation_date=datetime.date(1991, 12, 31))

Cell type: IncrementalCell
Previous Evaluation Date: 1991-12-31


### 2.3 Ledger-Specific Conventions: Dev-Lag

In Ledger's convention, dev-lag 0 corresponds to period_end. This means the first evaluation is at the end of the accident period.

In [46]:
# Demonstrate dev-lag convention
print("Dev-Lag Convention Demonstration:")
print("For Ledger, dev-lag 0 = period_end")
print()

# Find cells with dev-lag 0 (evaluation_date = period_end)
dev_lag_0_cells = []
for cell in test_triangle:
    if cell.evaluation_date == cell.period_end:
        dev_lag_0_cells.append(cell)
        if len(dev_lag_0_cells) <= 3:  # Show first 3 examples
            print(f"Dev-lag 0 cell:")
            print(f"  Period: {cell.period_start} to {cell.period_end}")
            print(f"  Evaluation: {cell.evaluation_date}")
            print(f"  Dev-lag: 0 months (evaluation = period_end)")
            print()

print(f"Found {len(dev_lag_0_cells)} cells with dev-lag 0")

Dev-Lag Convention Demonstration:
For Ledger, dev-lag 0 = period_end

Dev-lag 0 cell:
  Period: 1992-01-01 to 1992-12-31
  Evaluation: 1992-12-31
  Dev-lag: 0 months (evaluation = period_end)

Dev-lag 0 cell:
  Period: 1993-01-01 to 1993-12-31
  Evaluation: 1993-12-31
  Dev-lag: 0 months (evaluation = period_end)

Dev-lag 0 cell:
  Period: 1994-01-01 to 1994-12-31
  Evaluation: 1994-12-31
  Dev-lag: 0 months (evaluation = period_end)

Found 28 cells with dev-lag 0


In [47]:
# add in that we can handle negative dev lags

In [48]:
# Show development progression for a single accident period
if len(test_triangle.periods) > 0:
    first_period = test_triangle.periods[0]
    print(f"Development progression for period {first_period[0]} to {first_period[1]}:")
    
    period_cells = []
    for cell in test_triangle:
        if cell.period_start == first_period[0] and cell.period_end == first_period[1]:
            period_cells.append(cell)
    
    # Sort by evaluation date
    period_cells.sort(key=lambda x: x.evaluation_date)
    
    for i, cell in enumerate(period_cells[:5]):  # Show first 5 evaluations
        days_diff = (cell.evaluation_date - cell.period_end).days
        months_diff = days_diff // 30  # Approximate months
        print(f"  Evaluation {i+1}: {cell.evaluation_date} (dev-lag ~{months_diff} months)")
        if 'paid_loss' in cell.values:
            print(f"    Paid Loss: ${cell.values['paid_loss']:,.0f}")
    
    if len(period_cells) > 5:
        print(f"  ... and {len(period_cells)-5} more evaluations")

Development progression for period 1992-01-01 to 1992-12-31:
  Evaluation 1: 1992-12-31 (dev-lag ~0 months)
    Paid Loss: $40,386,429
  Evaluation 2: 1993-12-31 (dev-lag ~12 months)
    Paid Loss: $124,778,432
  Evaluation 3: 1994-12-31 (dev-lag ~24 months)
    Paid Loss: $249,719,443
  Evaluation 4: 1995-12-31 (dev-lag ~36 months)
    Paid Loss: $368,643,594
  Evaluation 5: 1996-12-31 (dev-lag ~48 months)
    Paid Loss: $477,857,831
  ... and 23 more evaluations


## 3. Triangle Slicing and Manipulation

Bermuda provides powerful slicing capabilities to extract subsets of triangle data.

### 3.1 Right Edge Analysis

The "right edge" represents the most recent evaluation for each accident period.

In [49]:
# Extract the right edge
right_edge = test_triangle.right_edge
print(f"Right edge contains {len(right_edge)} cells")
print("\nFirst few right edge cells:")
for i, cell in enumerate(right_edge[:3]):
    print(f"  Period {cell.period_start} to {cell.period_end}, evaluated on {cell.evaluation_date}")

Right edge contains 28 cells

First few right edge cells:
  Period 1992-01-01 to 1992-12-31, evaluated on 2019-12-31
  Period 1993-01-01 to 1993-12-31, evaluated on 2019-12-31
  Period 1994-01-01 to 1994-12-31, evaluated on 2019-12-31


In [50]:
# Plot the right edge data completeness
right_edge_completeness = right_edge.plot_data_completeness()
right_edge_completeness

alt.Chart(...)

### 3.2 Time-based Slicing

We can slice triangles by specific periods or evaluation dates.

In [51]:
# Slice by specific accident period (if we have 2010 data)
if len(test_triangle.periods) > 10:
    # Try to find a period around 2010 or use the 10th period
    target_period = test_triangle.periods[10]
    period_slice = test_triangle[target_period[0], :, :]
    print(f"Slice for period {target_period[0]} to {target_period[1]}:")
    print(f"Contains {len(period_slice)} evaluations")
    
    # Plot this slice
    period_completeness = period_slice.plot_data_completeness()
    period_completeness

Slice for period 2002-01-01 to 2002-12-31:
Contains 18 evaluations


### 3.3 Clipping Operations

Clipping allows us to restrict the triangle to specific date ranges or development periods.

In [52]:
# Get the evaluation range for clipping demonstration
eval_dates = test_triangle.evaluation_dates
if len(eval_dates) > 20:
    # Clip to exclude the most recent evaluations
    clip_date = eval_dates[-10]  # 10th from last
    clipped_triangle = test_triangle.clip(max_eval=clip_date)
    
    print(f"Original triangle: {len(test_triangle)} cells")
    print(f"Clipped triangle (max eval {clip_date}): {len(clipped_triangle)} cells")
    
    # Show the difference in completeness
    clipped_completeness = clipped_triangle.plot_data_completeness()
    clipped_completeness
else:
    print("Triangle has limited evaluation dates - skipping clipping demonstration")

Original triangle: 406 cells
Clipped triangle (max eval 2010-12-31): 190 cells


## 4. Filtering and Custom Operations

Bermuda allows for sophisticated filtering based on cell properties.

In [26]:
# Filter for significant losses (if paid_loss field exists)
if 'paid_loss' in test_triangle.fields:
    # Find cells with paid loss > median
    all_paid_losses = [cell['paid_loss'] for cell in test_triangle if 'paid_loss' in cell.values and cell['paid_loss'] is not None]
    if all_paid_losses:
        median_loss = sorted(all_paid_losses)[len(all_paid_losses)//2]
        print(f"Median paid loss: ${median_loss:,.0f}")
        
        large_loss_triangle = test_triangle.filter(lambda cell: cell.get('paid_loss', 0) > median_loss)
        print(f"Cells with above-median losses: {len(large_loss_triangle)} of {len(test_triangle)}")
        
        # Plot the filtered data
        if len(large_loss_triangle) > 0:
            large_loss_completeness = large_loss_triangle.plot_data_completeness()
            large_loss_completeness
else:
    print("No paid_loss field found for filtering demonstration")

Median paid loss: $236,246,114


AttributeError: 'CumulativeCell' object has no attribute 'get'

## 5. Checking for Disjoint Triangles

Disjoint triangles have gaps in their data. Let's examine this characteristic.

In [27]:
# Check triangle regularity
print(f"Triangle category: {test_triangle.category}")
print(f"Is regular triangle: {test_triangle.category == 'Regular'}")
print(f"Is disjoint: {test_triangle.category in ['Irregular', 'Erratic']}")

# For demonstration, create a disjoint triangle by filtering
if len(test_triangle) > 100:
    # Create an artificially sparse triangle
    sparse_triangle = test_triangle.filter(lambda cell: hash(str(cell.period_start)) % 3 == 0)
    print(f"\nCreated sparse triangle: {len(sparse_triangle)} cells (vs {len(test_triangle)} original)")
    print(f"Sparse triangle category: {sparse_triangle.category}")
    
    # Plot the sparse triangle to show disjoint nature
    sparse_completeness = sparse_triangle.plot_data_completeness()
    sparse_completeness

AttributeError: 'Triangle' object has no attribute 'category'

## 6. Advanced Visualization

Let's explore some of the built-in visualization capabilities.

In [28]:
# Right edge progression chart
if len(test_triangle.right_edge) > 5:
    right_edge_chart = test_triangle.plot_right_edge()
    right_edge_chart

In [53]:
test_triangle.slices

{Metadata(risk_basis='Accident', country=None, currency=None, reinsurance_basis='Gross', loss_definition='Loss+DCC', per_occurrence_limit=None, details={'line_of_busines': 'GL'}, loss_details={}):        Cumulative Triangle 
 
 
  Number of slices:  1 
  Number of cells:  406 
  Triangle category:  Regular 
  Experience range:  1992-01-01/2019-12-31 
  Experience resolution:  12 
  Evaluation range:  1992-12-31/2019-12-31 
  Evaluation resolution:  12 
  Dev Lag range:  0.0 - 324.0 months 
  Fields: 
    paid_loss
    reported_loss
  Optional Fields: 
    earned_premium (73.9% coverage)
  Common Metadata: 
    risk_basis  Accident 
    reinsurance_basis  Gross 
    loss_definition  Loss+DCC 
    line_of_busines  GL 
 }

In [29]:
# If we have a smaller triangle suitable for detailed visualization
if len(test_triangle) < 1000 and len(test_triangle) > 50:
    try:
        # Mountain chart shows development patterns
        mountain_chart = test_triangle.plot_mountain()
        mountain_chart
    except Exception as e:
        print(f"Mountain chart not available: {e}")

    try:
        # Ballistic chart shows loss development trajectories  
        ballistic_chart = test_triangle.plot_ballistic()
        ballistic_chart
    except Exception as e:
        print(f"Ballistic chart not available: {e}")
else:
    print(f"Triangle size ({len(test_triangle)} cells) not suitable for detailed visualization")

Triangle size (5442 cells) not suitable for detailed visualization


## 7. Multi-Slice Triangles

If our triangle contains multiple slices (different segments), we can analyze them separately.

In [54]:
# Examine triangle slices
print(f"Number of slices: {len(test_triangle.slices)}")

if len(test_triangle.slices) > 1:
    print("\nSlice details:")
    for i, slice_obj in enumerate(test_triangle.slices[:3]):  # Show first 3 slices
        print(f"  Slice {i+1}: {len(slice_obj)} cells")
        if hasattr(slice_obj, 'metadata') and slice_obj.metadata:
            print(f"    Metadata: {dict(slice_obj.metadata)}")
    
    # Plot multi-slice right edge if available
    try:
        multi_slice_chart = test_triangle.plot_right_edge()
        multi_slice_chart
    except Exception as e:
        print(f"Multi-slice visualization not available: {e}")
else:
    print("Triangle contains a single slice")
    print(f"Common metadata: {dict(test_triangle.common_metadata)}")

Number of slices: 1
Triangle contains a single slice


TypeError: 'Metadata' object is not iterable

## Summary

This notebook has demonstrated the key concepts of the Bermuda library:

1. **Triangles** - The main data structure for loss development data
2. **Cells** - Individual data points that can be cumulative or incremental
3. **Slicing** - Extracting subsets of data by period, evaluation date, or custom criteria
4. **Clipping** - Restricting triangles to specific date ranges
5. **Filtering** - Applying custom logic to select cells
6. **Data Completeness** - Understanding the shape and coverage of your data
7. **Ledger Conventions** - Dev-lag 0 = period_end for consistent development measurement
8. **Disjoint Triangles** - Handling irregular or sparse data patterns

The test triangle data provides a real-world example of how these concepts work with actual insurance loss development data.